## 1. 字典研究


### 1.1 用户评论 user reviews

- rating	浮动型（float）	产品评分（范围从 1.0 到 5.0）。
- title	字符串（str）	用户评论的标题。
- text	字符串（str）	用户评论的正文。
- images	列表（list）	用户在收到产品后发布的图片。每张图片有不同的尺寸（小、中、大），分别由 small_image_url、medium_image_url 和 large_image_url 表示。
- asin	字符串（str）	产品的 ID。
- parent_asin	字符串（str）	产品的父 ID。注意：通常不同颜色、款式、尺寸的产品属于同一个父 ID。以前的 Amazon 数据集中的 asin 实际上是父 ID。请使用父 ID 来查找产品的元数据。
- user_id	字符串（str）	评论者的 ID。
- timestamp	整数型（int）	评论时间（Unix 时间戳）。
- verified_purchase	布尔型（bool）	用户是否验证过购买。
- helpful_vote	整数型（int）	评论的有用票数。

### 1.2 商品元数据 Item Metadata

- main_category	字符串（str）	产品的主类目（即领域）。
- title	字符串（str）	产品名称。
- average_rating	浮动型（float）	产品页面显示的平均评分。
- rating_number	整数型（int）	产品的评分数量。
- features	列表（list）	产品的特点，以项目符号格式列出。
- description	列表（list）	产品的描述。
- price	浮动型（float）	产品的价格（以美元为单位）。
- images	列表（list）	产品的图片。每张图片有不同的尺寸（缩略图、大图、高分辨率图）。variant 字段显示图片的版本。
- videos	列表（list）	产品的视频，包括标题和 URL。
- store	字符串（str）	产品的商店名称。
- categories	列表（list）	产品的分层类别。
- details	字典（dict）	产品详情，包括材料、品牌、尺寸等。
- parent_asin	字符串（str）	产品的父 ID。
- bought_together	列表（list）	网站推荐的搭配商品。

In [121]:
from datasets import load_dataset
import torch
from transformers import BertTokenizer, BertModel
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import pandas as pd
import os

In [2]:
# 加载数据集
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Amazon_Fashion",
                       trust_remote_code=True, cache_dir='./data/Amazon_Fashion/')
meta_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Amazon_Fashion", split='full',
                            trust_remote_code=True, cache_dir='./data/Amazon_Fashion/')

In [5]:
print(meta_dataset[0]['images']['large'])

['https://m.media-amazon.com/images/I/41+cCfaVOFS._AC_.jpg', 'https://m.media-amazon.com/images/I/41jBdP7etRS._AC_.jpg', 'https://m.media-amazon.com/images/I/41UGJiRe7UL._AC_.jpg', 'https://m.media-amazon.com/images/I/41zb4GR-lWS._AC_.jpg', 'https://m.media-amazon.com/images/I/612BT4t-uFL._AC_.jpg', 'https://m.media-amazon.com/images/I/51ExLGv3QwL._AC_.jpg', 'https://m.media-amazon.com/images/I/313iU0xDEkS._AC_.jpg']


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# 初始化 BERT 模型和 tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", cache_dir='./data/bert-base-uncased')
bert_model = BertModel.from_pretrained("bert-base-uncased", cache_dir='./data/bert-base-uncased-model')
bert_model.eval().to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
# 初始化 ResNet 模型
resnet = models.resnet50(weights=True)
resnet.fc = torch.nn.Identity()  # 去除全连接层，只保留特征提取部分
resnet.eval().to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# 图像预处理
img_preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [11]:
# 流式处理文本特征
def extract_text_features(texts, batch_size=64, device='cuda'):
    features = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        # 转换输入文本为 BERT 需要的格式
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=128)
        
        # 确保所有输入张量都在同一个设备上
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 使用 BERT 模型
        with torch.no_grad():
            outputs = bert_model(**inputs)
        
        # 使用CLS token作为文本特征
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu()
        features.append(cls_embeddings)
    
    return torch.cat(features, dim=0)

In [12]:
# 流式处理图像特征
def extract_image_feature(image_url, device='cuda'):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        img_tensor = img_preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = resnet(img_tensor).cpu()
        return features.squeeze()
    except:
        return torch.zeros(2048)  # 空图像返回零向量

In [13]:
# 逐批处理数据
def process_data(dataset, meta_dataset, batch_size=64):
    all_text_features = []
    all_image_features = []
    
    # 处理数据集中的文本和图像
    for i in range(0, len(dataset), batch_size):
        # 处理文本
        batch_reviews = dataset[i:i+batch_size]['text']
        text_features = extract_text_features(batch_reviews)
        all_text_features.append(text_features)
        
    # 处理商品元数据中的图像
    for i in range(0, len(meta_dataset), batch_size):
        # 提取 large 图像
        batch_meta = meta_dataset[i:i+batch_size]
        image_urls = [meta['images']['large'] for meta in batch_meta if 'images' in meta and 'large' in meta['images']]
        image_features = extract_image_features(image_urls)
        all_image_features.append(image_features)
    
    # 将处理后的数据保存到文件
    save_to_csv(all_text_features, all_image_features)

In [14]:
# 处理数据并保存
def process_and_save_data(dataset, meta_dataset, output_dir="./data/processed_data"):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)

    # 保存用户商品信息的 CSV 文件
    user_item_info_file = os.path.join(output_dir, "user_item_info.csv")
    with open(user_item_info_file, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ["user_id", "asin", "parent_asin", "rating", "timestamp", "helpful_vote"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        # 处理前10000条评论（可调整）
        for i in range(10000):
            review = dataset['full'][i]
            meta = meta_dataset[i]

            # 提取文本特征
            text_features = extract_text_features([review['text']], batch_size=64, device='cuda')

            # 提取图像特征（如果有图像URL）
            image_features = []
            if 'images' in meta and 'large' in meta['images']:
                image_urls = meta['images']['large']
                image_features = extract_image_feature(image_urls[0], device='cuda')  # 只取第一个图像

            # 写入用户和商品的信息
            writer.writerow({
                "user_id": review['user_id'],
                "asin": review['asin'],
                "parent_asin": review['parent_asin'],
                "rating": review['rating'],
                "timestamp": review['timestamp'],
                "helpful_vote": review['helpful_vote']
            })

            # 保存文本特征和图像特征
            # 存储文本特征为 .npy 文件
            text_feature_file = os.path.join(output_dir, f"text_features_{i}.npy")
            np.save(text_feature_file, text_features.squeeze().cpu().numpy())

            # 存储图像特征为 .npy 文件
            image_feature_file = os.path.join(output_dir, f"image_features_{i}.npy")
            np.save(image_feature_file, image_features.cpu().numpy())

            # 打印处理进度
            if (i + 1) % 100 == 0:
                print(f"Processed {i + 1}/10000 records")

    print(f"Processed data saved to {output_dir}")

In [15]:
# 测试处理单条数据
review = dataset['full'][0]
meta = meta_dataset[0]

text_features = extract_text_features([review['text']], batch_size=1, device='cuda')
image_features = []
if 'images' in meta and 'large' in meta['images']:
    image_urls = meta['images']['large']
    image_features = extract_image_feature(image_urls[0], device='cuda')

# 打印处理的特征
print("Text Features:", text_features, text_features.shape)
print("Image Features:", image_features, image_features.shape)

Text Features: tensor([[-1.5794e-01, -3.0994e-01,  5.1385e-01, -3.3866e-01, -1.0661e-01,
         -2.8660e-01,  3.2009e-01,  6.5162e-01,  3.3201e-01, -4.5616e-01,
          4.2836e-02, -2.2227e-01,  1.5335e-01,  4.6260e-01, -8.6974e-02,
          2.2171e-01,  6.1301e-02,  7.0427e-01,  2.5363e-01,  4.0587e-02,
         -2.4557e-01, -5.5593e-01,  5.3485e-01, -2.8614e-01,  2.0127e-02,
          6.9144e-03, -3.4372e-01,  2.5355e-01,  5.4387e-02,  1.8149e-02,
          1.6128e-01,  3.0740e-01, -5.3399e-01, -2.7201e-02,  9.6023e-01,
         -9.0721e-02, -4.7360e-03,  5.2853e-02,  6.9606e-02, -4.9508e-02,
         -2.9004e-01,  8.2307e-02,  4.2140e-01, -5.1297e-02, -4.1493e-02,
         -5.5871e-01, -4.1468e+00,  1.9847e-01, -1.6316e-01, -2.3715e-01,
          3.3993e-01, -5.1091e-01, -2.8263e-01,  4.3078e-01,  6.1901e-01,
          4.4071e-01, -8.2627e-01,  1.1231e-01,  8.6912e-02, -1.3843e-01,
          8.8562e-02, -1.9257e-01, -5.8624e-02,  1.6097e-01,  2.7554e-01,
          2.9568e-01, -

In [16]:
# 开始处理并保存数据
process_and_save_data(dataset, meta_dataset)

Processed 100/10000 records
Processed 200/10000 records
Processed 300/10000 records
Processed 400/10000 records
Processed 500/10000 records
Processed 600/10000 records
Processed 700/10000 records
Processed 800/10000 records
Processed 900/10000 records
Processed 1000/10000 records
Processed 1100/10000 records
Processed 1200/10000 records
Processed 1300/10000 records
Processed 1400/10000 records
Processed 1500/10000 records
Processed 1600/10000 records
Processed 1700/10000 records
Processed 1800/10000 records
Processed 1900/10000 records
Processed 2000/10000 records
Processed 2100/10000 records
Processed 2200/10000 records
Processed 2300/10000 records
Processed 2400/10000 records
Processed 2500/10000 records
Processed 2600/10000 records
Processed 2700/10000 records
Processed 2800/10000 records
Processed 2900/10000 records
Processed 3000/10000 records
Processed 3100/10000 records
Processed 3200/10000 records
Processed 3300/10000 records
Processed 3400/10000 records
Processed 3500/10000 re

## 2. 利用特征数据进行建模和推荐算法设计

In [149]:
from torch_geometric.data import Data
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# 加载 CSV 文件
df = pd.read_csv("./data/processed_data/user_item_info.csv")

In [19]:
df.head()

,user_id,asin,parent_asin,rating,timestamp,helpful_vote
0,AGBFYI2DDIKXC5Y4FARTYDTQBMFQ,B00LOPVX74,B00LOPVX74,5.0,1578528394489,3
1,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B07B4JXK8D,B07B4JXK8D,5.0,1608426246701,0
2,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,B007ZSEQ4Q,B007ZSEQ4Q,2.0,1432344828000,3
3,AFVNEEPDEIH5SPUN5BWC6NKL3WNQ,B07F2BTFS9,B07F2BTFS9,1.0,1546289847095,2
4,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,B00PKRFU4O,B00XESJTDE,5.0,1439476166000,0


In [23]:
# 使用LabelEncoder对用户ID和商品ID进行编码
user_encoder = LabelEncoder()
df['user_idx'] = user_encoder.fit_transform(df['user_id'])

product_encoder = LabelEncoder()
df['product_idx'] = product_encoder.fit_transform(df['asin'])

In [24]:
df.head()

,user_id,asin,parent_asin,rating,timestamp,helpful_vote,user_idx,product_idx
0,AGBFYI2DDIKXC5Y4FARTYDTQBMFQ,B00LOPVX74,B00LOPVX74,5.0,1578528394489,3,2349,562
1,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B07B4JXK8D,B07B4JXK8D,5.0,1608426246701,0,1789,3724
2,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,B007ZSEQ4Q,B007ZSEQ4Q,2.0,1432344828000,3,3561,194
3,AFVNEEPDEIH5SPUN5BWC6NKL3WNQ,B07F2BTFS9,B07F2BTFS9,1.0,1546289847095,2,1946,4131
4,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,B00PKRFU4O,B00XESJTDE,5.0,1439476166000,0,3857,701


In [90]:
# 创建边（edges）
edge_index = torch.tensor([df['user_idx'].values, df['product_idx'].values + df['user_idx'].nunique()], dtype=torch.long)

edge_label = torch.tensor(df['rating'].values, dtype=torch.float32)

# 打印边的信息
print("Edge index:", edges.shape)
print("Edge label:", edge_label.shape)

Edge index: (2, 10000)
Edge label: torch.Size([10000])


In [91]:
# 设定文件夹路径
data_dir = './data/processed_data'

# 获取文件夹内所有的 .npy 文件名
text_files = sorted([f for f in os.listdir(data_dir) if 'text_features' in f and f.endswith('.npy')])
image_files = sorted([f for f in os.listdir(data_dir) if 'image_features' in f and f.endswith('.npy')])

# 初始化用于存储特征的列表
text_features_list = []
image_features_list = []

# 加载所有的 text_features 和 image_features
for text_file, image_file in zip(text_files, image_files):
    text_feature = np.load(os.path.join(data_dir, text_file))  # 加载文本特征
    image_feature = np.load(os.path.join(data_dir, image_file))  # 加载图像特征
    
    text_features_list.append(text_feature)
    image_features_list.append(image_feature)

# 将所有的特征合并为一个大的 numpy 数组
text_features = np.stack(text_features_list, axis=0)
image_features = np.stack(image_features_list, axis=0)

print(f"Text features shape: {text_features.shape}")
print(f"Image features shape: {image_features.shape}")

Text features shape: (10000, 768)
Image features shape: (10000, 2048)


In [93]:
# 合并 text_features、image_features 和 user_features
item_features = np.concatenate([text_features, image_features], axis=1)

In [94]:
item_features.shape

(10000, 2816)

In [95]:
# 构造 user 节点的特征
num_users = df['user_idx'].nunique()
user_features = np.zeros((num_users, item_features.shape[1]))

In [96]:
user_features.shape

(4086, 2816)

In [97]:
# 拼接 user 和 item 节点特征
x = torch.tensor(np.vstack([user_features, item_features]), dtype=torch.float32)

In [98]:
def extract_time_features(df, ts_col='timestamp'):
    # 转换为 datetime 格式（注意是毫秒）
    df['dt'] = df[ts_col].apply(lambda x: datetime.fromtimestamp(x / 1000))
    
    # 提取年（减去最小年份后再归一化）
    df['year'] = df['dt'].dt.year
    df['year'] = (df['year'] - df['year'].min()) / (df['year'].max() - df['year'].min())

    # 月（1~12）
    df['month'] = (df['dt'].dt.month - 1) / 11.0

    # 周（0=Monday, 6=Sunday）
    df['weekday'] = df['dt'].dt.weekday / 6.0

    # 小时（0~23）
    df['hour'] = df['dt'].dt.hour / 23.0

    return df[['year', 'month', 'weekday', 'hour']]

In [99]:
# 1. 提取边的时间特征
time_features = extract_time_features(df)  # df 为你的边信息 DataFrame

In [100]:
# 2. 提取 helpful_votes
helpful = df['helpful_vote'].values.reshape(-1, 1)
helpful = (helpful - helpful.min()) / (helpful.max() - helpful.min())  # 归一化

In [101]:
# 3. 拼接所有边特征
edge_attr = np.hstack([time_features.values, helpful])
edge_attr = torch.tensor(edge_attr, dtype=torch.float)

In [150]:
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, edge_label=edge_label)

In [151]:
data.x.shape, data.edge_index.shape, data.edge_attr.shape, data.edge_label.shape

(torch.Size([14086, 2816]),
 torch.Size([2, 15996]),
 torch.Size([10000, 5]),
 torch.Size([10000]))

In [152]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [166]:
def prepare_data_split(data, val_ratio=0.1, test_ratio=0.1, seed=42):
    assert hasattr(data, 'edge_label'), "data.edge_label 不存在，无法划分训练集"
    
    num_edges = data.edge_label.size(0)
    indices = list(range(num_edges))

    # 1. 划分 train / val+test
    train_idx, temp_idx = train_test_split(indices, test_size=val_ratio + test_ratio, random_state=seed)
    # 2. 再划分 val / test
    val_size = val_ratio / (val_ratio + test_ratio)
    val_idx, test_idx = train_test_split(temp_idx, test_size=1 - val_size, random_state=seed)

    # 3. 创建划分后的边索引、特征、标签
    train_idx = torch.tensor(train_idx)
    val_idx = torch.tensor(val_idx)
    test_idx = torch.tensor(test_idx)

    data.train_pos_edge_index = data.edge_index[:, train_idx]
    data.train_edge_attr = data.edge_attr[train_idx]
    data.train_edge_label = data.edge_label[train_idx]

    data.val_pos_edge_index = data.edge_index[:, val_idx]
    data.val_edge_attr = data.edge_attr[val_idx]
    data.val_edge_label = data.edge_label[val_idx]

    data.test_pos_edge_index = data.edge_index[:, test_idx]
    data.test_edge_attr = data.edge_attr[test_idx]
    data.test_edge_label = data.edge_label[test_idx]

    print(f"数据划分完成：Train {len(train_idx)}, Val {len(val_idx)}, Test {len(test_idx)}")
    return data

In [167]:
data = prepare_data_split(data)

数据划分完成：Train 8000, Val 1000, Test 1000


In [168]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, edge_feat_dim, dropout=0.3):
        super().__init__()
        # 两层 GCN
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = nn.LayerNorm(hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = nn.LayerNorm(hidden_channels)

        self.dropout = dropout

        # 用于边上的评分预测：拼接两端节点表示 + 边特征
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * hidden_channels + edge_feat_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        # 节点嵌入
        x = self.gcn1(x, edge_index)
        x = self.norm1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gcn2(x, edge_index)
        x = self.norm2(x)

        # 提取每条边两端的节点表示
        src, dst = edge_index  # 两端的索引
        edge_input = torch.cat([x[src], x[dst], edge_attr], dim=1)

        # 预测评分
        return self.edge_mlp(edge_input).view(-1)

In [173]:
model = GCN(in_channels=2816, hidden_channels=256, edge_feat_dim=5).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [174]:
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    # 前向传播：使用训练边
    out = model(data.x, data.train_pos_edge_index, data.train_edge_attr)

    # 计算损失
    loss = criterion(out, data.train_edge_label)
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def evaluate(model, data, criterion):
    model.eval()

    # 验证集
    val_out = model(data.x, data.val_pos_edge_index, data.val_edge_attr)
    val_loss = criterion(val_out, data.val_edge_label).item()
    val_rmse = val_loss ** 0.5

    # 测试集
    test_out = model(data.x, data.test_pos_edge_index, data.test_edge_attr)
    test_loss = criterion(test_out, data.test_edge_label).item()
    test_rmse = test_loss ** 0.5

    return val_rmse, test_rmse

In [178]:
for epoch in range(1, 2001):
    loss = train(model, data, optimizer, criterion)
    if epoch % 100 == 0:
        val_rmse, test_rmse = evaluate(model, data, criterion)
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val RMSE: {val_rmse:.4f} | Test RMSE: {test_rmse:.4f}")

Epoch 100 | Loss: 0.6405 | Val RMSE: 1.4487 | Test RMSE: 1.3866
Epoch 200 | Loss: 0.3767 | Val RMSE: 1.5022 | Test RMSE: 1.4324
Epoch 300 | Loss: 0.2483 | Val RMSE: 1.5054 | Test RMSE: 1.4380
Epoch 400 | Loss: 0.1625 | Val RMSE: 1.4799 | Test RMSE: 1.4252
Epoch 500 | Loss: 0.1284 | Val RMSE: 1.4862 | Test RMSE: 1.4376
Epoch 600 | Loss: 0.1179 | Val RMSE: 1.4970 | Test RMSE: 1.4464
Epoch 700 | Loss: 0.1043 | Val RMSE: 1.4962 | Test RMSE: 1.4517
Epoch 800 | Loss: 0.0971 | Val RMSE: 1.4958 | Test RMSE: 1.4538
Epoch 900 | Loss: 0.0857 | Val RMSE: 1.4960 | Test RMSE: 1.4596
Epoch 1000 | Loss: 0.0834 | Val RMSE: 1.4963 | Test RMSE: 1.4581
Epoch 1100 | Loss: 0.0787 | Val RMSE: 1.5170 | Test RMSE: 1.4638
Epoch 1200 | Loss: 0.0720 | Val RMSE: 1.5118 | Test RMSE: 1.4709
Epoch 1300 | Loss: 0.0679 | Val RMSE: 1.4988 | Test RMSE: 1.4562
Epoch 1400 | Loss: 0.0622 | Val RMSE: 1.5203 | Test RMSE: 1.4763
Epoch 1500 | Loss: 0.0629 | Val RMSE: 1.5026 | Test RMSE: 1.4558
Epoch 1600 | Loss: 0.0596 | Val RM